# QGAR - A Flashcard Generating NLP Model

This notebook shows how to load and use the `QGAR` model.

Please read the [README](./readme.md) before continueing!

**Table of Contents:**
1. [Load QGAR](#load-qgar)
2. [Download and Preprocess SQuAD Dataset](#download-and-preprocess-squad-dataset)
3. [Run QGAR](#run-qgar)
4. [Train QGAR](#4-train-qgar)

</br>

---

</br>

## 1. Load QGAR
First, we must load the `QGAR` model and tokenizer.

In [ ]:
import torch
from huggingface_hub import login
from main import load_model_and_tokenizer, get_hg_token

login(get_hg_token(), add_to_git_credential=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model, tokenizer = load_model_and_tokenizer("t5-small", device)

## 2. Download and Preprocess SQuAD Dataset

First, we download and preprocess the modified `SQuAD` dataset, adding separator (`<sep>`) and end of sequence tokens (`<\s>`) to each entry.
The preprocessed file is split in two datasets, `training` and `validation`, and the sets are saved in the `data` directory in `PyTorch` format under the names:
* [training_data.pt](./data/training_data.pt)
* [validation_data.pt](./data/validation_data.pt)

In [ ]:
from preprocessing.preprocessor import Preprocessor

preprocessor = Preprocessor(model, tokenizer)
preprocessor.preprocess_dataset()

## 3. Run QGAR
Next, we import `QGAR` and set up a pipeline.
Now, we can simply pass a context to the model to generate questions.

In [1]:
%load_ext autoreload

%autoreload 2

from models.qg import QG
import json

qt = QG()
questions = qt("Historical Fiction is one of those sub-genres of literature that takes many forms. It's most important feature, though, is that it's set in the past, with every element of the story conforming to the norms of the day. Here's how we define Historical Fiction, a look at its origins, and some popular types.")
print(json.dumps(questions, indent=4))

2023-03-19 17:20:05.228125: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


OSError: the-coorporation/t5-qg does not appear to have a file named config.json. Checkout 'https://huggingface.co/the-coorporation/t5-qg/main' for available files.

## 4. Train QGAR
To train `QGAR`, we first parse `settings.json` to get the training arguments.

We then make an instance of `QGARTrainer` and call `train` which will train the model and push it to `The Coorporation`'s Huggingface Hub.

In [ ]:
%load_ext autoreload
%autoreload 2

from main import parse_settings
from models.qg import QG

%env WANDB_PROJECT=t5-qg

model_args, data_args, train_args = parse_settings()

with open("./.local/wandb_token.txt", "r") as f:
    key = f.read()

qt = QG("t5-small", "t5-small")
qt.train(train_args, data_args, key)

</br>

---

# 5. Answer Generator

In [ ]:
%load_ext autoreload

%autoreload 2

from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

question_answers = []

print(questions)

for question in questions:
    result = question_answerer(question=question, context=input_text)
    question_answers.append({ "question": question, "answer": result["answer"], "score": result["score"] })

print(question_answers)